In [1]:
import numpy as np
import copy
import sys
import random

init_board = [[0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0],
              [0,0,0,1,2,0,0,0],
              [0,0,0,2,1,0,0,0],
              [0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0]]

mv_vect = [[-1, -1], [0, -1], [1, -1], [-1, 0], [1, 0], [-1, 1], [0, 1], [1, 1]]

board = [[]]

def score_board(board):
    w_score = 0
    b_score = 0
    
    for y in range(8):
        for x in range(8):
            if (board[x][y] == 1):
                w_score += 1
            elif (board[x][y] == 2):
                b_score += 1
    return (w_score, b_score)

def draw_board(board):
    HLINE = '  +---+---+---+---+---+---+---+---+'
    VLINE = '  |   |   |   |   |   |   |   |   |'

    print '    A   B   C   D   E   F   G   H'
    print HLINE
    for y in range(8):
        print y+1,
        for x in range(8):
            c = ' '
            if (board[x][y] == 1):
                c = 'W'
            elif (board[x][y] == 2):
                c = 'B'
            print '| %s' % c, 
        print '|'
        print HLINE
        
def chk_bnds(square):
    return square[0] <= 7 and square[0] >= 0 and square[1] <= 7 and square[1] >= 0 

def get_color(board, square):
    return board[square[0]][square[1]]

def flip_squares(board, flips, square, player):
    new_board = copy.deepcopy(board)  
        
    for f in flips:
        lo = square
        new_board[lo[0]][lo[1]] = player
        lo = np.add(lo, f)
        while(get_color(board, lo) != player):
            new_board[lo[0]][lo[1]] = player
            lo = np.add(lo, f)
            
                
    return new_board

def eval_move(board, square, player):
    flips = []
    
    i = -1
    
    for mv in mv_vect:
        i += 1
        curr_square = square
        curr_square = np.add(curr_square, mv)
        if (not chk_bnds(curr_square) or get_color(board, curr_square) == player or get_color(board, curr_square) == 0):
            continue
        while (chk_bnds(curr_square)):   
            if(get_color(board, curr_square) == player):
                flips.append(mv)
                pass
            curr_square = np.add(curr_square, mv)
    if (len(flips) > 0 ):
        return [True, flip_squares(board, flips, square, player)]
    return [False, board]


def get_moves(board, player):
    moves = []
    cnd = []
    for y in range(8):
        for x in range(8):
            if (board[x][y] == 0):
                cnd = eval_move(board, [x,y], player)
                if (cnd[0]):
                    moves.append(cnd[1])
    if (len(moves) == 0):
        return [board]
    else:
        return moves

In [2]:
def select_move(board, player):
    best_score = 9999
    best_move = []
    depth = 2
    p2 = 2
    moves = get_moves(board, player)
    
    if (player == 2):
        p2 = 1
    for m in moves:
        nextp_moves = get_moves(m, p2)
        nextp_bs = 0
        for np_m in nextp_moves:
            sc = score_board(np_m)[p2 - 1]
            if (sc > nextp_bs):
                nextp_bs = sc
        if (nextp_bs < best_score):
            best_score = nextp_bs
            best_move = m
    return best_move

def rand_move(board, player):
    moves = get_moves(board, player)
    r = random.randint(0,len(moves) - 1)
    return moves[r]

def human_move(board, player):
    nums = ['1','2','3','4','5','6','7','8'] 
    letters = ['a','b','c','d','e','f','g','h']
    if (get_moves(board, player) == [board]):
        print "No valid moves, your turn is skipped."
    while True:
        print 'Please enter your move. In the form letter then number.'
        user_in = raw_input().lower()
        move = list(user_in)
        if (len(move) == 2 and move[1] in nums and move[0] in letters):
            cnd = eval_move(board, [ord(move[0]) - 97, int(move[1] ) -1, ], player)
            if (cnd[0]):
                return cnd[1]
                break
            else:
                print "Not a valid move."
            
            
    

In [6]:
def play_game(p1, p2, b):
      
    if (p1 == 1):
        mmv =  select_move(init_board, 2)
    elif (p1 == 2):
        mmv =  rand_move(init_board, 2)
    else:
        mmv = human_move(init_board, 2)
    i = 0
    
    while (True):
        i += 1
        if (b == 1):
            draw_board(mmv)
        if (p2 == 1):
            mmv =  select_move(mmv, 1)
        elif (p2 == 2):
            mmv =  rand_move(mmv, 1)
        else:
            mmv = human_move(mmv, 1)
        
        if (get_moves(mmv, 1) == get_moves(mmv, 2) == [mmv]):
            break
        
            
        if (b == 1):
            draw_board(mmv)
        
        if (p1 == 1):
            mmv =  select_move(mmv, 2)
        elif (p2 == 2):
            mmv =  rand_move(mmv, 2)
        else:
            mmv = human_move(mmv, 2)
            
        if (get_moves(mmv, 1) == get_moves(mmv, 2) == [mmv]):
            break
            
    score = score_board(mmv)
    print 'Player 1 scored', score[1]
    print 'Player 2 scored', score[0]
        
    if (score[1] > score[0]):
        return 1
    elif (score[1] < score[0]):
        return 2
    else:
        return 0

In [7]:
def game_start():
    p1 = 0
    p2 = 0
    b = 0
    while True:
        print '1 for AI player 1, 2 for random player 1, and 3 for human player 1'
        user_in = raw_input().lower()
        if (user_in == '1'):
            p1 = 1
            break
        elif (user_in == '2'):
            p1 = 2
            break
        elif (user_in == '3'):
            p1 = 3
            break
        else:
            print 'Command not recognized'
    while True:
        print '1 for AI player 2, 2 for random player 2, and 3 for human player 2'
        user_in = raw_input().lower()
        if (user_in == '1'):
            p2 = 1
            break
        elif (user_in == '2'):
            p2 = 2
            break
        elif (user_in == '3'):
            p2 = 3
            break
        else:
            print 'Command not recognized'
    while True:
        print '1 for board, 2 to disable'
        user_in = raw_input().lower()
        if (user_in == '1'):
            b = 1
            break
        elif (user_in == '2'):
            b = 2
            break
        else:
            print 'Command not recognized'
    play_game(p1, p2, b)

In [8]:
p1_tot = 0
p2_tot = 0
for i in range(300):
    r = play_game(1, 2, 2)
    if (  r == 1 ):
        p1_tot += 1
    elif (r == 2):
        p2_tot += 1

print "Player 1 won ", p1_tot
print "Player 2 won", p2_tot

Player 1 scored 40
Player 2 scored 24
Player 1 scored 53
Player 2 scored 11
Player 1 scored 51
Player 2 scored 13
Player 1 scored 30
Player 2 scored 34
Player 1 scored 53
Player 2 scored 11
Player 1 scored 49
Player 2 scored 15
Player 1 scored 35
Player 2 scored 29
Player 1 scored 24
Player 2 scored 40
Player 1 scored 25
Player 2 scored 39
Player 1 scored 39
Player 2 scored 25
Player 1 scored 49
Player 2 scored 15
Player 1 scored 41
Player 2 scored 22
Player 1 scored 37
Player 2 scored 27
Player 1 scored 31
Player 2 scored 0
Player 1 scored 46
Player 2 scored 18
Player 1 scored 21
Player 2 scored 43
Player 1 scored 42
Player 2 scored 0
Player 1 scored 26
Player 2 scored 38
Player 1 scored 20
Player 2 scored 44
Player 1 scored 21
Player 2 scored 0
Player 1 scored 34
Player 2 scored 30
Player 1 scored 33
Player 2 scored 31
Player 1 scored 9
Player 2 scored 55
Player 1 scored 41
Player 2 scored 23
Player 1 scored 23
Player 2 scored 0
Player 1 scored 46
Player 2 scored 18
Player 1 scored 3